In [1]:
import torch
import torch.nn as nn

In [2]:
class Base:

    def __init__(self, model, optimizer, loss):
        self.model = model
        self.optimizer = optimizer
        self.loss = loss

    def to(self, device):
        pass
    



    